# Team: Kayla and Hariraj

# Dataset3: Alcohol Sales

https://www.kaggle.com/datasets/bulentsiyah/for-simple-exercises-time-series-forecasting/data?select=Alcohol_Sales.csv

# Preamble

In [2]:
# import relevant Python libraries

import numpy as np
import pandas as pd
from scipy import stats as sps
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, Latex
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# A. Exploration of Dataset

# B. Pre Processing

# C. Feature Extraction/ Feature Engineering

# D. Processing of Dataset using two different machine learning methods

# E. Comparative Analysis of the two methods

# F. Discussion on ethical isues

# G. Bibliography